# Using a Modflow model as a stressmodel in Pastas

This notebook shows how to use a simple Modflow model as stress model in Pastas.

## Packages

In [ ]:
import os

import flopy
import pandas as pd
import pastas as ps
from pastas.timer import SolveTimer

import pastas_plugins.modflow as ppmf

ps.set_log_level("ERROR")

## Download MODFLOW executable

In [ ]:
bindir = "bin"
mf6_exe = os.path.join(bindir, "mf6.exe")
if not os.path.isfile(mf6_exe):
    if not os.path.isdir(bindir):
        os.makedirs(bindir)
    flopy.utils.get_modflow("bin", repo="modflow6")

## Data

In [ ]:
# %%
tmin = pd.Timestamp("2001-01-01")
tmax = pd.Timestamp("2014-12-31")

tmin_wu = tmin - pd.Timedelta(days=3651)
tmin_wu = pd.Timestamp("1986-01-01")

head = (
    pd.read_csv(
        "https://raw.githubusercontent.com/pastas/pastas/master/doc/examples/data/head_nb1.csv",
        index_col="date",
        parse_dates=True,
    )
    .squeeze()
    .loc[tmin:tmax]
)
prec = (
    pd.read_csv(
        "https://raw.githubusercontent.com/pastas/pastas/master/doc/examples/data/rain_nb1.csv",
        index_col="date",
        parse_dates=True,
    )
    .squeeze()
    .loc[tmin_wu:tmax]
)
evap = (
    pd.read_csv(
        "https://raw.githubusercontent.com/pastas/pastas/master/doc/examples/data/evap_nb1.csv",
        index_col="date",
        parse_dates=True,
    )
    .squeeze()
    .loc[tmin_wu:tmax]
)

ps.plots.series(head, [prec, evap], hist=False);

## Time series models

### Standard exponential model

In [ ]:
# %%
# create model with exponential response function
mlexp = ps.Model(head)
mlexp.add_stressmodel(
    ps.RechargeModel(prec=prec, evap=evap, rfunc=ps.Exponential(), name="test_exp")
)
mlexp.solve(tmin=tmin, tmax=tmax)
mlexp.plot();

### Uncalibrated MODFLOW time series model

Using parameters based on the Pastas Exponential model.

In [ ]:
# %%
# extract resistance and sy from exponential model
# transform exponential parameters to modflow resistance and sy
mlexp_c = mlexp.parameters.loc["test_exp_A", "optimal"]
mlexp_c_i = mlexp.parameters.loc["test_exp_A", "initial"]
mlexp_sy = (
    mlexp.parameters.loc["test_exp_a", "optimal"]
    / mlexp.parameters.loc["test_exp_A", "optimal"]
)
mlexp_sy_i = (
    mlexp.parameters.loc["test_exp_a", "initial"]
    / mlexp.parameters.loc["test_exp_A", "initial"]
)
mlexp_d = mlexp.parameters.loc["constant_d", "optimal"]
mlexp_d_i = mlexp.parameters.loc["constant_d", "initial"]
mlexp_f = mlexp.parameters.loc["test_exp_f", "optimal"]
mlexp_f_i = mlexp.parameters.loc["test_exp_f", "initial"]

In [ ]:
# create modflow pastas model with c and sy
mlexpmf = ps.Model(head)
# shorten the warmup to speed up the modflow calculation somewhat.
mlexpmf.settings["warmup"] = pd.Timedelta(days=4 * 365)
expmf = ppmf.ModflowRch(exe_name=mf6_exe, sim_ws="mf_files/test_expmf")
expsm = ppmf.ModflowModel([prec, evap], modflow=expmf, name="test_expmfsm")
mlexpmf.add_stressmodel(expsm)
mlexpmf.set_parameter(f"{expsm.name}_sy", initial=mlexp_sy, vary=False)
mlexpmf.set_parameter(f"{expsm.name}_c", initial=mlexp_c, vary=False)
mlexpmf.set_parameter(f"{expsm.name}_f", initial=mlexp_f, vary=False)
mlexpmf.set_parameter("constant_d", initial=mlexp_d, vary=False)
# mlexpmf.solve()
mlexpmf.plot(tmin=head.index[0]);

## Calibrated MODFLOW time series model

Now fit a Pastas Model using the Modflow model as a response function. This takes some
time, as the modflow model has to be recomputed for every iteration in the optimization
process.

In [ ]:
ml = ps.Model(head)
# shorten the warmup to speed up the modflow calculation somewhat.
ml.settings["warmup"] = pd.Timedelta(days=4 * 365)
mf = ppmf.ModflowRch(exe_name=mf6_exe, sim_ws="mf_files/test_mfrch")
sm = ppmf.ModflowModel([prec, evap], modflow=mf, name="test_mfsm")
ml.add_stressmodel(sm)
ml.set_parameter(f"{sm.name}_sy", initial=mlexp_sy_i, vary=True)
ml.set_parameter(f"{sm.name}_c", initial=mlexp_c_i, vary=True)
ml.set_parameter(f"{sm.name}_f", initial=mlexp_f_i, vary=True)
ml.set_parameter("constant_d", initial=mlexp_d_i, vary=True)

with SolveTimer() as st:
    ml.solve(callback=st.timer, fit_constant=False)

In [ ]:
ml.plot();

## Results

### Parameters

In [ ]:
ml.parameters.style.set_table_attributes('style="font-size: 12px"').set_caption(
    "Pastas-Modflow"
)

In [ ]:
mlexp.parameters.style.set_table_attributes('style="font-size: 12px"').set_caption(
    "Pastas-Exponential"
)

Compare parameters from the Pastas-Modflow model to the "true" parameters derived from
the Pastas exponential model.

In [ ]:
comparison = pd.DataFrame(
    {
        "True": mlexpmf.parameters["initial"].values,
        "MF6": ml.parameters["optimal"].values,
    },
    index=ml.parameters.index,
)
comparison["Difference"] = comparison["MF6"] - comparison["True"]
comparison["% Difference"] = (comparison["Difference"] / comparison["True"]) * 100
comparison.style.format(precision=2)

### Plots

Compare the Pastas-Modflow simulation to the Pastas-Exponential simulation.

In [ ]:
ax = ml.plot()  # Pastas-Modflow
mlexp.plot(ax=ax);  # Pastas-Exponential